# Batch run models
1. Running model
2. Evaluating results

In [ ]:
%load_ext lab_black
import os, json
import pandas as pd
import altair as alt
from meta import check_cfgs_params
from evaluate import make_df_wnw

In [ ]:
batch_name = "O2P_m270_last3ticks"
batch_output_dir = "batch_eval/{}/".format(batch_name)

## Plotting

Review the batch structure

In [ ]:
cfgs = pd.read_csv(batch_output_dir + 'cfgs.csv', index_col=0)
df = pd.read_csv(batch_output_dir + 'bcdf.csv', index_col=0)
check_cfgs_params(cfgs)

### Explicitly provide varying h-params after reviewing unique params

In [ ]:
variates = ['hidden_units', 'learning_rate', 'p_noise']

# View h-param grid

In [ ]:
cfgs

In [ ]:
def count_models(cfgs):
    print(
        'There are {} models in the datafile'.format(
            len(df.code_name.unique())
        )
    )
    
    # Select useful data
    sel_df = df.loc[(df.timestep == df.timestep.max()) &
                    df.cond.isin(conditions), variates + ['cond', 'epoch', dv]]

    dfm = df[['code_name'] + variates].pivot_table(index='code_name')
    dfm['code_name'] = dfm.index

    pvt = dfm.pivot_table(
        index=['p_noise', 'hidden_units', 'learning_rate', 'cleanup_units'],
        aggfunc='count',
        values='code_name'
    )

    pvt.reset_index(inplace=True)
    pvt.rename(columns={'code_name': 'n'}, inplace=True)

    plot_n = alt.Chart(pvt).mark_rect().encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color="n:O",
        tooltip=[
            "p_noise", "hidden_units", "cleanup_units", "learning_rate", "n"
        ],
    ).properties(title="Model counts")

    return plot_n



In [ ]:
def plot_std(df, variates, conditions, dv):
    """
    Plot standard deviation of VARIABLE at given CONDITIONS at LAST TIME STEP
    df: pandas dataframe containing batch condition data file (bcdf)
    variates: a list of varying hyperparameters
    conditions: filter by conditions at df.cond
    dv: dependent variable to plot on heatmap (e.g., acc, sse)
    """
    # Select useful data
    sel_df = df.loc[(df.timestep == df.timestep.max()) &
                    df.cond.isin(conditions), variates + ['cond', 'epoch', dv]]

    # Calculate standard deveiation in each cell
    plot_df = sel_df.groupby(variates + ['epoch']).std().reset_index()

    # Plot heatmap
    return alt.Chart(plot_df).mark_rect().encode(
        x="epoch:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column='p_noise:O',
        color=alt.Color(dv, scale=alt.Scale(domain=[0, 0.2])),
        tooltip=variates + [dv],
    )

In [ ]:


df = df.pivot_table(index=variates + ['epoch', 'timestep', 'cond']
                   ).reset_index()

In [ ]:
alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

########## Overview heatmap model selector ##########
df_ov = df[(df.epoch == df.epoch.max()) & (df.timestep == df.timestep.max())]

overview = (
    alt.Chart(df_ov).mark_rect().encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color=alt.Color(
            "mean(acc):Q",
            scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["acc"] + variates,
    ).add_selection(sel_run).properties(
        title="Overall accuracy (incl. all words and nonwords)"
    )
)

overview

### Generate unique setting ID to replace code_name

In [ ]:
df_setting_str = map(
    lambda x, y, z: str(x) + '_' + str(y) + '_' + str(z), df.hidden_units,
    df.p_noise, df.learning_rate
)

df['code_name'] = list(df_setting_str)

In [ ]:
########## Word vs. Nonword accuracy over epoch ##########
df_wnw = make_df_wnw(
    df, word_cond=['INC_HF'], nonword_cond=['unambiguous']
)  # Match with Rueckl

In [ ]:
cfgs_setting_str = map(
    lambda x, y, z: str(x) + '_' + str(y) + '_' + str(z), cfgs.hidden_units,
    cfgs.p_noise, cfgs.learning_rate
)

cfgs['code_name'] = list(cfgs_setting_str)

In [ ]:
df_wnw = df_wnw.merge(cfgs)

In [ ]:
mdf_wnw = df_wnw.melt(
    id_vars=['code_name', 'epoch'],
    value_vars=['word_acc', 'nonword_acc'],
    var_name='wnw',
    value_name='acc'
)

plot_epoch = alt.Chart(mdf_wnw).mark_point(size=80).encode(
    y=alt.Y("mean(acc):Q", scale=alt.Scale(domain=(0, 1))),
    x="epoch:Q",
    color=alt.Color("code_name", legend=None),
    shape="wnw:N",
    opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
    tooltip=["code_name", "epoch", "acc"],
).add_selection(sel_run).properties(
    title="Plot word and nonword accuracy by epoch"
)

########## Word vs. Nonword plot with diagonal ##########

wnw_line = alt.Chart(df_wnw).mark_line().encode(
    y=alt.Y("mean(nonword_acc):Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color("code_name", legend=None),
    opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
    tooltip=["code_name", "epoch", "word_acc", "nonword_acc"] + variates,
)

wnw_point = wnw_line.mark_point().encode(
    color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen")),
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

wnw = diagonal + wnw_line + wnw_point

wnw_interactive = wnw.add_selection(sel_run).properties(
    title="Word vs. Nonword accuracy at final time step"
)

dashboard = overview | plot_epoch | wnw_interactive
dashboard.save(batch_output_dir + 'dashboard.html')
dashboard

Main effect plot

In [ ]:
def main_effect_plot(df, var):

    pdf = df.pivot_table(
        index=['epoch', var], values=['word_acc', 'nonword_acc']
    )

    pdf.reset_index(inplace=True)
    pdf['word_advantage'] = pdf.word_acc - pdf.nonword_acc

    sel_run = alt.selection(type="multi", on="click", fields=[var])

    overview_wacc = alt.Chart(pdf).mark_rect().encode(
        y=alt.Y(var, type="ordinal"),
        x='epoch:O',
        color=alt.Color(
            "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["word_acc", "nonword_acc"],
    ).add_selection(sel_run).properties(
        title="Word - Nonword accuracy (word accuracy) heatmap by {} and epoch".
        format(var)
    )

    overview_wadv = overview_wacc.encode(
        color=alt.Color(
            "word_advantage",
            scale=alt.Scale(scheme="redyellowgreen", domain=(-.2, .2))
        )
    ).properties(
        title="Word - Nonword accuracy (word advantage) heatmap by {} and epoch"
        .format(var)
    )

    wnw_line = alt.Chart(pdf).mark_line().encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color(var, type="ordinal", scale=alt.Scale(scheme="magma")),
        opacity=alt.condition(sel_run, alt.value(0.9), alt.value(0)),
        tooltip=[var, "epoch", "word_acc", "nonword_acc"],
    )

    diagonal = alt.Chart(pd.DataFrame({
        'x': [0, 1],
        'y': [0, 1]
    })).mark_line(color='black').encode(x='x', y='y')

    return overview_wacc | overview_wadv | (diagonal + wnw_line)


p = alt.vconcat()
for v in variates:
    p &= main_effect_plot(df_wnw, v)

p.save(batch_output_dir + 'main_effects.html')
p

In [ ]:
df_wnw['word_advantage'] = df_wnw.word_acc - df_wnw.nonword_acc

In [ ]:
base = alt.Chart(df_wnw).mark_point().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "epoch", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 100))
    ),
    opacity=alt.value(0.2),
    tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

plot = diagonal + base

plot.save(batch_output_dir + 'all_models_wnw.html')

In [ ]:
base = alt.Chart(df_wnw).mark_rect().encode(
    x="p_noise:O",
    y="hidden_units:O",
    row="learning_rate:O",
    column="epoch:O",
    tooltip=[
        "p_noise", "hidden_units", "cleanup_units", "learning_rate", "word_acc",
        "nonword_acc"
    ],
)

w = base.encode(
    color=alt.
    Color("word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1)))
).properties(title="Word acc")

w.save(batch_output_dir + 'word.html')

In [ ]:
nw = base.encode(
    color=alt.Color(
        "nonword_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Nonword acc")
nw.save(batch_output_dir + 'nonword.html')

In [ ]:
adv = base.encode(
    color=alt.Color(
        "word_advantage",
        scale=alt.Scale(scheme="redyellowgreen", domain=(-.3, .3))
    )
).properties(title="Word advantage")

adv.save(batch_output_dir + 'wadv.html')

In [ ]:
df_agg = df_wnw.pivot_table(
    index=['epoch', 'learning_rate', 'hidden_units', 'p_noise']
)

df_agg.reset_index(inplace=True)
df_agg

In [ ]:
plot_pnoise = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color("p_noise", type="ordinal", scale=alt.Scale(scheme="reds")),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

p = alt.layer(diagonal + plot_pnoise, data=df_agg).facet(
    row="hidden_units:O", column="learning_rate:O"
)

p.save(batch_output_dir + 'p_noise.html')

In [ ]:
plot_hidden = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "hidden_units", type="ordinal", scale=alt.Scale(scheme="blues")
    ),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

h = alt.layer(diagonal + plot_hidden,
              data=df_agg).facet(row="p_noise:O", column="learning_rate:O")

h.save(batch_output_dir + 'hidden.html')

In [ ]:
plot_lr = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "learning_rate", type="ordinal", scale=alt.Scale(scheme="greens")
    ),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

lr = alt.layer(diagonal + plot_lr,
               data=df_agg).facet(row="hidden_units:O", column="p_noise:O")

lr.save(batch_output_dir + 'lr.html')

### Variance dignostic

In [ ]:
variates = ['hidden_units', 'learning_rate', 'p_noise']

plot_std(df, variates, ['INC_HF'],
         'acc').save(batch_output_dir + 'stdev_strain_INCHF.html')

plot_std(df, variates, ['unambiguous'],
         'acc').save(batch_output_dir + 'stdev_grain_unambiguous.html')

taraban_w = [
    'High-frequency exception', 'High-frequency regular-inconsistent',
    'Low-frequency exception', 'Low-frequency regular-inconsistent',
    'Regular control for High-frequency exception',
    'Regular control for High-frequency regular-inconsistent',
    'Regular control for Low-frequency exception',
    'Regular control for Low-frequency regular-inconsistent'
]

plot_std(df, variates, taraban_w,
         'acc').save(batch_output_dir + 'stdev_taraban_all.html')

glushko_nw = ['Exception', 'Regular']
plot_std(df, variates, glushko_nw,
         'acc').save(batch_output_dir + 'stdev_glushko.html')